## Hybrid Search Langchain

In [1]:
from google.colab import userdata
api_key=userdata.get("PINECONE_API_KEY")

In [10]:
! pip install langchain_community langchain pinecone pinecone-text pinecone-notebooks langchain_huggingface

In [3]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [7]:
import os
from pinecone import Pinecone, ServerlessSpec
index_name="hybrid-search-langchain"
#initialize Pinecone Client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
  pc.create_index(
    name=index_name,
    dimension=384, #dimensionality of dense model
    metric="dotproduct", #sparse values supported only for dotproduct
    spec=ServerlessSpec(cloud="aws", region="us-east-1"),
  )

In [8]:
index=pc.Index(index_name)
index

In [12]:
# vector embedding and sparse matrix
from google.colab import userdata
userdata.get('HF_TOKEN')

from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
from pinecone_text.sparse import BM25Encoder #this encoder uses TF-IDF by default

bm25_encoder=BM25Encoder().default()
bm25_encoder

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
sentences=[
    "In 2023, I ate Pizza",
    "In 2022, I ate Burger",
    "In 2021, I ate Chicken"
]

#Applyin TF-IDF values on these sentences
bm25_encoder.fit(sentences)

#store the values to a json file
bm25_encoder.dump("bm25_encoder_values.json")

#load the BM25Encoder Object
bm25_encoder=BM25Encoder().load("bm25_encoder_values.json")

  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
#Retriever
retriever=PineconeHybridSearchRetriever(
    embeddings=embeddings,
    sparse_encoder=bm25_encoder,
    index=index
)

In [16]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x78bf9df7f610>, index=<pinecone.db_data.index.Index object at 0x78c14c549450>)

In [17]:
retriever.add_texts([
    "In 2023, I ate Pizza",
    "In 2022, I ate Burger",
    "In 2021, I ate Chicken"
])

  0%|          | 0/1 [00:00<?, ?it/s]

In [19]:
retriever.invoke("What I ate First")

[Document(metadata={'score': 0.27839613}, page_content='In 2021, I ate Chicken'),
 Document(metadata={'score': 0.244397402}, page_content='In 2022, I ate Burger'),
 Document(metadata={'score': 0.236842632}, page_content='In 2023, I ate Pizza')]